In [12]:
import pandas
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

In [22]:
model = torchvision.models.resnet152(weights=True).cuda()

c:\Users\leomo\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
normalize = transforms.Compose(
    [transforms.Resize(256),
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])

image = torchvision.datasets.ImageNet(root='data', split='val', transform=normalize)

In [25]:

val_loader = torch.utils.data.DataLoader(image, batch_size=128, shuffle=False, num_workers=2)


In [26]:
# accuracy
model.eval()
correct = 0
total = 0
for i, (images, labels) in enumerate(val_loader):
    images = images.cuda()
    labels = labels.cuda()
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()



In [27]:
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 78 %
